# Project: [Analyzing Unicorn Companies](https://app.datacamp.com/learn/projects/1531)

Did you know that the average return from investing in stocks is 10% per year! But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

`dates`

| Column       | Description                                 |
|--------------|---------------------------------------------|
| company_id   | A unique ID for the company.                |
| date_joined  | The date that the company became a unicorn. |
| year_founded | The year that the company was founded.      |

`funding`

| Column           | Description                                 |
|------------------|---------------------------------------------|
| company_id       | A unique ID for the company.                |
| valuation        | Company value in US dollars.                |
| funding          | The amount of funding raised in US dollars. |
| select_investors | A list of key investors in the company.     |

`industries`

| Column     | Description                                |
|------------|--------------------------------------------|
| company_id | A unique ID for the company.               |
| industry   | The industry that the company operates in. |

`companies`

| Column     | Description                                       |
|------------|---------------------------------------------------|
| company_id | A unique ID for the company.                      |
| company    | The name of the company.                          |
| city       | The city where the company is headquartered.      |
| country    | The country where the company is headquartered.   |
| continent  | The continent where the company is headquartered. |


# Instructions
Your task is to first identify the three best-performing industries based on the number of new unicorns created over the last three years (2019, 2020, and 2021) combined.

From there, you will write a query to return the industry, the year, the number of companies in these industries that became unicorns each year in 2019, 2020, and 2021, along with the average valuation per industry per year, converted to billions of dollars and rounded to two decimal places!

As the firm is interested in trends for the top-performing industries, your results should be displayed by industry, then year in descending order.

The final output of your query will look like this:

| **industry**	| **year**	| **num_unicorns**	| **average_valuation_billions** | 
| :--- |:--- | --- | --- |
| industry1	| 2021	| ---	| --- | 
| industry1	| 2020	| ---	| --- | 
| industry1	| 2019	| ---	| --- | 
| industry2	| 2021	| ---	| --- | 
| industry2	| 2020	| ---	| --- | 
| industry2	| 2019	| ---	| --- | 
| industry3	| 2021	| ---	| --- | 
| industry3	| 2020	| ---	| --- | 
| industry3	| 2019	| ---	| --- | 

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.


For this we'll used [DuckDB](https://duckdb.org/) 0.9.2 with [JupySQL](https://jupysql.ploomber.io/en/latest/quick-start.html) 0.10.5. The original project uses a PostgreSQL database called `unicorns` which we'll replicate in memory with DuckDB.

### Setup our environment and load the data
**Note:** The output displayed will be that of a Pandas DataFrame instead of a result set. This was done so that the output is more like that of a GUI. 

In [1]:
%load_ext sql
%sql duckdb:// --alias unicorns
%config SqlMagic.autopandas = True
%config SqlMagic.displaycon = False

Connecting to 'unicorns'

Create our tables and load in the data.

In [2]:
%sql DROP TABLE IF EXISTS companies;
%sql CREATE TABLE companies AS SELECT * FROM read_csv_auto('data/unicorns_companies.csv', types={'company_id': 'INTEGER', 'company': 'VARCHAR', 'city': 'VARCHAR', 'country': 'VARCHAR', 'continent': 'VARCHAR'});

%sql DROP TABLE IF EXISTS dates;
%sql CREATE TABLE dates AS SELECT * FROM read_csv_auto('data/unicorns_dates.csv', types={'company_id': 'INTEGER', 'date_joined': 'DATE','year_founded': 'INTEGER'});

%sql DROP TABLE IF EXISTS funding;
%sql CREATE TABLE funding AS SELECT * FROM read_csv_auto('data/unicorns_funding.csv', types={'company_id': 'INTEGER', 'valuation': 'BIGINT','funding': 'BIGINT', 'select_investors': 'VARCHAR'});

%sql DROP TABLE IF EXISTS industries;
%sql CREATE TABLE industries AS SELECT * FROM read_csv_auto('data/unicorns_industries.csv', types={'company_id': 'INTEGER', 'industry': 'VARCHAR'});

,Success


# Breaking down the problem into high-level small steps

1. Identify the three best-performing industries based on the number of new `unicorns created` over the last three years (`2019`, `2020`, and `2021`) combined.
2. Write a query to return the `industry`, the `year`, the `number of companies` in these `industries` that became unicorns each year in `2019`, `2020`, and `2021`, along with the average valuation per `industry per year`, converted to _billions of dollars and rounded to two decimal places_!
3. Display the results as per the output above. 



First off lets take a quick peek at the data types for the tables we've been provide by looking at the information_schema.columns for each table.

In [3]:
%%sql unicorns
SELECT table_name, column_name, data_type 
FROM information_schema.columns
WHERE table_name IN ('companies', 'dates', 'funding', 'industries')
ORDER By table_name, column_name;

,table_name,column_name,data_type
0,companies,city,VARCHAR
1,companies,company,VARCHAR
2,companies,company_id,INTEGER
3,companies,continent,VARCHAR
4,companies,country,VARCHAR
5,dates,company_id,INTEGER
6,dates,date_joined,DATE
7,dates,year_founded,INTEGER
8,funding,company_id,INTEGER
9,funding,funding,BIGINT


`year_joined` - is the only column with a `date` data type  
`columnd_id` - is present in each of the tables and has `integer` as its data type which will make it perfect for joins

Let's take a quick look at the `dates` table:

In [4]:
%%sql unicorns
SELECT * FROM dates
LIMIT 3;

,company_id,date_joined,year_founded
0,189,2017-06-24,1919
1,848,2021-06-01,2019
2,556,2022-02-15,2011


We'll need to modify the output of date_joined to get in a format that we can used for filtering as we're only interested in the year portion. We wil use [EXTRACT](https://duckdb.org/docs/sql/functions/timestamptz) function to perform this operation. 

In [5]:
%%sql unicorns
SELECT
    date_joined,
    EXTRACT(YEAR FROM date_joined) AS year
FROM dates
LIMIT 3;

,date_joined,year
0,2017-06-24,2017
1,2021-06-01,2021
2,2022-02-15,2022


Next we need to combine the `dates` table with the `industries` table and join them using the `company_id`. 

We're only interested in the `2019`, `2020` and `2021` years so will filter on those values. 

We also want to want a `count` of the number of companies that joined those `industries` so that we can determine which are top 3.

In [6]:
%%sql unicorns
-- Query for determining the industries with the most number of companies that joined each industry
-- Ordered by most to last and limited to 3.
SELECT 
  COUNT(i.company_id) as num_companies,
  industry
FROM industries AS i
INNER JOIN dates AS d
ON i.company_id = d.company_id
WHERE EXTRACT(YEAR FROM date_joined) IN ('2019', '2020', '2021')
GROUP BY industry
ORDER BY num_companies DESC
LIMIT 3;

,num_companies,industry
0,173,Fintech
1,152,Internet software & services
2,75,E-commerce & direct-to-consumer


Great! The top 3 industries by number of companies that joined in the combined years of 2019, 2020 and 2021 are:
- **Fintech**
- **Internet software & services** 
- **E-commerce & direct-to-consumer**

Now we need to breakdown step 2 into smaller manageble pieces:
> Write a query to return the industry, the year, the number of companies in these industries that became unicorns each year in 2019, 2020, and 2021, along with the average valuation per industry per year, converted to billions of dollars and rounded to two decimal places!

Lets start small by focusing on just one industry `Fintech`.

We'll need to:
1. Count the number of companies that joined that industry for each year, 2019, 2020 and 2021.
2. We can grab the average valuation at the same time to save some work.

The tables we will need are `industries`, `funding` and `dates`.

- From `industries` we need to select the `industry` field so we can pull back only those companies that listed as `Fintech`.
- From `funding` we need to select the `valuation` field we we can calculate the average valuation. 
- From `dates` we need to select the `date_joined` field so we can filter for only those companies that joined in 2019, 2020 and 2021.

First lets confirm that we get 173 rows when we filter on `Fintech` after all our joins.

In [7]:
%%sql unicorns
-- First lets confirm that we get 173 rows when we filter on 'Fintech'
SELECT
  i.company_id,
  i.industry,
  f.valuation,
  EXTRACT(YEAR FROM d.date_joined)
FROM industries AS i
INNER JOIN dates AS d
  ON i.company_id = d.company_id
INNER JOIN funding AS f
  ON i.company_id = f.company_id
WHERE EXTRACT(YEAR FROM d.date_joined) IN ('2019', '2020', '2021')
AND i.industry = 'Fintech'

,company_id,industry,valuation,"main.date_part('year', d.date_joined)"
0,848,Fintech,1000000000,2021
1,396,Fintech,2000000000,2021
2,983,Fintech,1000000000,2019
3,844,Fintech,1000000000,2021
4,894,Fintech,1000000000,2021
...,...,...,...,...
168,271,Fintech,3000000000,2021
169,664,Fintech,1000000000,2021
170,146,Fintech,5000000000,2021
171,651,Fintech,1000000000,2021


Great, now lets add our `count`, `avg` and `grouping`.

In [8]:
%%sql unicorns
SELECT
  i.industry AS industry,
  EXTRACT(YEAR FROM date_joined) AS year,
  COUNT(i.company_id) AS num_unicorns,
  ROUND(
    AVG(f.valuation) / 1000000000 :: NUMERIC
    , 2) AS average_valuation_billions
FROM industries AS i
INNER JOIN dates AS d
  ON i.company_id = d.company_id
INNER JOIN funding AS f
  ON i.company_id = f.company_id
WHERE EXTRACT(YEAR FROM date_joined) IN ('2019', '2020', '2021')
AND i.industry = 'Fintech'
GROUP BY industry, year
ORDER BY industry, year DESC;

,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Fintech,2020,15,4.33
2,Fintech,2019,20,6.80


That looks good. Now lets pull in all of the top 3 industries into the query using a CTE and JOIN based off of the `top_industries` query we used previous. 

In [9]:
%%sql unicorns
WITH top_industries AS (
  SELECT
    COUNT(i.company_id) as num_companies,
    i.industry
  FROM industries AS i
  INNER JOIN dates AS d
	  ON i.company_id = d.company_id
  WHERE EXTRACT(YEAR FROM d.date_joined) IN ('2019', '2020', '2021')
  GROUP BY i.industry
  ORDER BY num_companies DESC
  LIMIT 3
)
SELECT
  i.industry AS industry,
  EXTRACT(YEAR FROM d.date_joined) AS year,
  COUNT(i.company_id) AS num_unicorns,
  ROUND(
    AVG(f.valuation) / 1000000000 :: NUMERIC
    , 2) AS average_valuation_billions
FROM industries AS i
INNER JOIN dates AS d
  ON i.company_id = d.company_id
INNER JOIN funding AS f
  ON i.company_id = f.company_id
INNER JOIN top_industries as ti -- this will filter on the top three industries we care about
  ON i.industry = ti.industry
WHERE EXTRACT(YEAR FROM d.date_joined) IN ('2019', '2020', '2021')
GROUP BY i.industry, year
ORDER BY i.industry, year DESC;

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23


And here's our final result in markdown as above:
| industry                        | year | num_unicorns | average_valuation_billions |
|:--------------------------------|:-----|-------------:|---------------------------:|
| E-commerce & direct-to-consumer | 2021 |           47 |                       2.47 |
| E-commerce & direct-to-consumer | 2020 |           16 |                          4 |
| E-commerce & direct-to-consumer | 2019 |           12 |                       2.58 |
| Fintech                         | 2021 |          138 |                       2.75 |
| Fintech                         | 2020 |           15 |                       4.33 |
| Fintech                         | 2019 |           20 |                        6.8 |
| Internet software & services    | 2021 |          119 |                       2.15 |
| Internet software & services    | 2020 |           20 |                       4.35 |
| Internet software & services    | 2019 |           13 |                       4.23 |

***
Below is the answer that **DataCamp** provides which uses two CTE's and a subquery to get to the final result. 

The `yearly_rankings` CTE seems unnecessary as the results from that query returns 124 rows and generates averages for industries and dates outside of the industries and years of interest. This could have larger effects if there were many more rows.

A few other things based on their query:
1. The `AVG(average_valuation / 1000000000)` portion in the final SELECT query seems redundant as it will attempt to take an Average of an already averaged value and as we're already grouping by year so it's just a division by 1.
```sql
ROUND(AVG(average_valuation / 1000000000), 2) AS average_valuation_billions
```
I believe that the line should have been:
```sql
ROUND(average_valuation / 1000000000, 2) AS average_valuation_billions
``````
2. The `GROUP BY industry, num_unicorns, year, average_valuation` is totally unneccesary when corrected to just using the ROUND without the additional AVG function.



**Datacamp Solution**

In [12]:
%%sql unicorns
-- DataCamp Query Solution
WITH top_industries AS
(
    SELECT i.industry, 
        COUNT(i.*) AS count
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    WHERE EXTRACT(year FROM d.date_joined) in ('2019', '2020', '2021')
    GROUP BY industry
    ORDER BY count DESC
    LIMIT 3
),

yearly_rankings AS 
(
    SELECT COUNT(i.*) AS num_unicorns,
        i.industry,
        EXTRACT(year FROM d.date_joined) AS year,
        AVG(f.valuation) AS average_valuation
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    INNER JOIN funding AS f
        ON d.company_id = f.company_id
    GROUP BY industry, year
)

SELECT industry,
    year,
    num_unicorns,
    ROUND(AVG(average_valuation / 1000000000), 2) AS average_valuation_billions
FROM yearly_rankings
WHERE year in ('2019', '2020', '2021')
    AND industry in (SELECT industry
                    FROM top_industries)
GROUP BY industry, num_unicorns, year, average_valuation
ORDER BY industry, year DESC;

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23


In [11]:
%%sql unicorns
-- yearly_rankings query from above
SELECT COUNT(i.*) AS num_unicorns,
	i.industry,
	EXTRACT(year FROM d.date_joined) AS year,
	AVG(f.valuation) AS average_valuation
FROM industries AS i
INNER JOIN dates AS d
	ON i.company_id = d.company_id
INNER JOIN funding AS f
	ON d.company_id = f.company_id
GROUP BY industry, year

,num_unicorns,industry,year,average_valuation
0,119,Internet software & services,2021,2.151261e+09
1,47,E-commerce & direct-to-consumer,2021,2.468085e+09
2,4,Mobile & telecommunications,2015,1.500000e+09
3,2,Fintech,2015,5.500000e+09
4,6,E-commerce & direct-to-consumer,2016,6.666667e+09
...,...,...,...,...
119,2,Hardware,2022,2.500000e+09
120,1,Auto & transportation,2015,2.000000e+09
121,1,Auto & transportation,2014,7.000000e+09
122,3,Travel,2021,2.666667e+09


**Modified Datacamp Solution**

In [10]:
%%sql unicorns
-- Modified DataCamp Query Solution to remove redundancy in final SELECT to see if we get the same solution in the end.
WITH top_industries AS
(
    SELECT i.industry, 
        COUNT(i.*) AS count
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    WHERE EXTRACT(year FROM d.date_joined) in ('2019', '2020', '2021')
    GROUP BY industry
    ORDER BY count DESC
    LIMIT 3
),

yearly_rankings AS -- this could probably go away or be improved upon.
(
    SELECT COUNT(i.*) AS num_unicorns,
        i.industry,
        EXTRACT(year FROM d.date_joined) AS year,
        AVG(f.valuation) AS average_valuation
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    INNER JOIN funding AS f
        ON d.company_id = f.company_id
	
    GROUP BY industry, year
)

SELECT industry,
    year,
    num_unicorns,
	ROUND(average_valuation / 1000000000, 2) AS average_valuation_billions
FROM yearly_rankings
WHERE year in ('2019', '2020', '2021')
    AND industry in (SELECT industry
                    FROM top_industries)
-- GROUP BY industry, num_unicorns, year, average_valuation
ORDER BY industry, year DESC;

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23
